# Practicalities of AutoML forecasting

In this notebook, we will walk through the process of trying to actually push a real world dataset through AutoML. AutoML has expectations of clean data which are rarely encountered with real world data. We will show various techniques of detecting the problems before they crash AutoML, and workarounds that will get you through the day.

Eventually, these tips and tricks will become part of the AutoML product, prettied up and made more robust.

Many of the cells in this notebook will show the first attempt at doing something, which is later abandoned or cleaned up. They are marked with `DEAD END`. Don't run them, they are there so that you can learn from my mistakes, not your.

To begin, let's load some data. Our first dead end occurs in the data loading, and is shown in the companion notebook `GetDataFromSQL`. But for now, _assume_ we have the data.

In [1]:
import pandas

SQLdata = pandas.read_csv('data/tutorial_data.csv.bz2')
SQLdata.describe(include = 'all')

,SalesDate,Item,Site,Channel,Quantity
count,1262885,1262885,1262885,1262885,1.262885e+06
unique,2057,348,11,9,NaN
top,2017-05-26,A53634,82DC,811B,NaN
freq,996,43416,359820,391703,NaN
mean,NaN,NaN,NaN,NaN,1.201671e+01
std,NaN,NaN,NaN,NaN,2.587775e+01
min,NaN,NaN,NaN,NaN,1.000000e+00
25%,NaN,NaN,NaN,NaN,5.000000e+00
50%,NaN,NaN,NaN,NaN,8.000000e+00
75%,NaN,NaN,NaN,NaN,1.200000e+01


The data is a friendly 1.2M rows, which is because it only contains the 348 items beginning with 'A'.

In [2]:
SQLdata['Item'].drop_duplicates()

0          A00968
377        A010AB
10431      A01370
10463      A02987
10778      A04247
10798      A04A18
16493      A04F8F
16583      A05356
18630      A05358
18806      A05794
20960      A06133
22241      A0627A
29656      A074CC
29702      A0802D
31102      A08952
31398      A08B54
32303      A09216
32341      A094CE
70554      A09781
70654      A09D61
71098      A0BC16
78938      A0BCF4
78998      A0BE42
79053      A0BF66
79469      A0CA5B
81591      A0CD71
84503      A0E454
86679      A0E9E8
86693      A0F431
86813      A0FA4E
            ...  
1093802    AE79B6
1093859    AE7C41
1100816    AE815D
1101168    AE8349
1103239    AE91A8
1103242    AE92B6
1103813    AEA5F7
1104110    AEBBB7
1106332    AEC7F3
1107898    AEDF2A
1107954    AEE6C5
1108399    AEF12C
1109364    AF0B7F
1109893    AF387A
1139718    AF3B02
1147859    AF5276
1147928    AF56E5
1148442    AF59E4
1148509    AF6FB8
1149349    AF7091
1149505    AF7676
1149621    AF88F0
1150516    AF8A0C
1178951    AF9EAB
1180540   

Now we'll establish the time series metadata. These say: we want to forecast the Quantity, along the time axis given by SalesDate, for each combination of Item, Site, and Channel.

In [3]:
grain_colnames = ['Item', 'Site', 'Channel']
time_colname = 'SalesDate'
target_colname = 'Quantity'

In [4]:
# How many series do we have?
guppy = SQLdata.groupby(grain_colnames)

print("Rows pulled : " + str(len(SQLdata)))
print("Distinct time series : " + str(len(guppy)))

Rows pulled : 1262885
Distinct time series : 6139


# Let's forecast!

AutoML in theory does not need much more than a data frame. So let's see how far AutoML gets on this **tough** dataset. Let us not even split into train and test (because we may need to re-work the data).

In [5]:
X_train = SQLdata.copy()
y_train = X_train.pop(target_colname).values

Begin our forecasting journey with the boilerplate AutoML imports and connections.

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import logging
import warnings
import os
# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None

import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [7]:
ws = Workspace.from_config()

# choose a name for the experiment in the workspace
# 'Experiment' roughly corresponds to a 'problem to solve'
experiment_name = 'automl-grocery'
# where to write models and other artifacts locally
project_folder = './sample_projects/automl-grocery'
experiment = Experiment(ws, experiment_name)

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [8]:
output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Location,westus2
Project Directory,./sample_projects/automl-grocery
Resource Group,automl
Run History Name,automl-grocery
SDK version,0.1.0.3119914
Subscription ID,938fa533-eeb9-4121-b97f-05b31c6eb088
Workspace,automl-customers


Now, we specify what our forecasting problem is, put the spec in the config object form, and run it.

In [9]:
time_series_settings = {
    'time_column_name': time_colname,
    'grain_column_names': grain_colnames,
    'drop_column_names': [],
    'max_horizon': 31
}

automl_config = AutoMLConfig(task='forecasting',
                             debug_log='automl-grocery.log',
                             primary_metric='normalized_root_mean_squared_error',
                             iterations=5,
                             X=X_train,
                             y=y_train,                             
                             n_cross_validations=2,
                             enable_ensembling=False,
                             path=project_folder,
                             verbosity=logging.INFO,    
                             **time_series_settings)

local_run = experiment.submit(automl_config, show_output=True)

Running on local machine


DataException: Frequencies can not be inferred for grain [('A00968', '33A1', '08BB')]. Please consider dropping that grain.

The errors are expected: our dataset has many intermittent series. For example, look at this series, focusing on the `SalesDate` column. Could *you* determine what its natural reporting frequency is?

In [10]:
SQLdata[(SQLdata['Item'] == 'A0CA5B') & (SQLdata['Site'] == '1B3B') & (SQLdata['Channel'] == '811B')]

,SalesDate,Item,Site,Channel,Quantity
79490,2012-01-24,A0CA5B,1B3B,811B,4.00
79491,2012-01-30,A0CA5B,1B3B,811B,9.00
79492,2012-02-02,A0CA5B,1B3B,811B,4.00
79493,2012-02-04,A0CA5B,1B3B,811B,3.00
79494,2012-02-17,A0CA5B,1B3B,811B,5.00
79495,2012-02-29,A0CA5B,1B3B,811B,4.00
79496,2012-03-07,A0CA5B,1B3B,811B,4.00
79497,2012-03-10,A0CA5B,1B3B,811B,7.00
79498,2012-03-14,A0CA5B,1B3B,811B,5.00
79499,2012-03-23,A0CA5B,1B3B,811B,5.00


# Technique 1: Filling in the implicit zeros

Let's help AutoML determine what the periodicity is by making the series explicitly daily. Since we are talking about sales, missing values correspond to zero quantity sold.

In [11]:
# We have a nice helpful function that fills out the data frame with:
# * zeros for the target column
# * NaNs for the rest of the data
# 
# Let's import it from out time series library in the repo.
from tslib.ts_functions import fill_out_with_zeros

In [12]:
# since frequency is hard to infer with intermittent data, we will tell the function explicitly
# the frequency is daily
complete = fill_out_with_zeros(SQLdata, time_colname, grain_colnames, target_colname, 'D')
complete.shape

Got 1262885 values NOT including zeros
Expecting 6063038 values including zeros
Check these match... otherwise bad things happen to merge.
['Item: object', 'Site: object', 'Channel: object', 'SalesDate: datetime64[ns]']
['Item: object', 'Site: object', 'Channel: object', 'SalesDate: datetime64[ns]']


(6063038, 1)

Note the data engorged to 6M rows - all the days with no record now have explicit 0 sales. We call that a "flat" dataframe.

In [13]:
# the data comes out with the grain and time in a pandas MultiIndex
flat_complete = complete.reset_index()
flat_complete.to_csv("grocery_flat.csv.bz2", compression='bz2', index=False)

In [14]:
# how do I split the data? It's about 5 years of data, 2012-01-01 to 2017-08-31.
# It probably does not make sense to do more than one month ahead of daily predictions
n_test_periods = 31 # days in August 2017

# this will split off the last n VALUES from each grain. If the data has implicit zeros,
# that means the test TIME INTERVAL may vary by grain.
from tslib.ts_functions import split_last_n_by_grain

# Do note the use of the parameter specifying the minimum grain length.
# Because of how we pulled the data from SQL, the use is redundant.
# But if your whole data fits in memory, you can filter out the short grains
# in python, rather than at the source, using the splitting function
df_train, df_test = split_last_n_by_grain(complete, 
                                          n_test_periods, 
                                          time_colname, grain_colnames, 
                                          min_grain_length=100)

In [15]:
print(df_train.shape)
print(df_test.shape)

(5872729, 1)
(190309, 1)


We know how many datapoints we have. But do we really know where the train and test sets are situated in time?
Let's look at their time extents. For ease of plotting, only two items (but with all their Site and Channel combinations) will be shown.

In [16]:
smaller = complete.loc[(['A00968', 'A09D61'], slice(None), slice(None), slice(None)), :]
small_train, small_test = split_last_n_by_grain(smaller, n_test_periods, time_colname, grain_colnames, min_grain_length=100)
print(small_train.shape)
print(small_test.shape)

# this is for checking the time extent of the grains
ranges_train = small_train.reset_index().groupby(grain_colnames).agg({'SalesDate': ['min', 'max']})
ranges_train.columns = ['begin', 'end']
ranges_train_flat = ranges_train.reset_index()

ranges_test = small_test.reset_index().groupby(grain_colnames).agg({'SalesDate': ['min', 'max']})
ranges_test.columns = ['begin', 'end']
ranges_test_flat = ranges_test.reset_index()

# we should be able to plot that
%matplotlib notebook
import matplotlib
matplotlib.rcParams['figure.figsize'] = [10, 12]
import matplotlib.pyplot as plt

plt.hlines(range(len(ranges_train_flat)),xmin=ranges_train_flat['begin'].values,xmax=ranges_train_flat['end'].values)
plt.hlines(range(len(ranges_test_flat)),xmin=ranges_test_flat['begin'].values,xmax=ranges_test_flat['end'].values, colors='r')
ytick_tuples = list(zip(ranges_train_flat['Item'],ranges_train_flat['Site'], ranges_train_flat['Channel']))
ytick_labels = list(map(lambda t : '-'.join(t), ytick_tuples))
plt.yticks(range(len(ranges_test_flat)), ytick_labels)
plt.show()

# Improvements on this plot could include 
#    * plot little series instead of horizontal lines, 
#    * color-separating zero values


(15948, 1)
(1023, 1)


<IPython.core.display.Javascript object>

We should ask ourselves whether the items whose histories do not reach the temporal end of the dataset are worth forecasting. Those items may have been discontinued, or they are still carried by the store but purchased infrequently. Since that is a question only a business owner can answer, let's leave them in for now. But have that little practicality in mind.

Let's save the dataset we have in case we need to restart from this point.

In [17]:
# Checkpoint: flatten for writing and save
flat_test = df_test.reset_index()
flat_test.to_csv("grocery_flat_test.csv.bz2", compression='bz2', index=False)

In [18]:
flat_train = df_train.reset_index()
flat_train.to_csv("grocery_flat_trainvalid.csv.bz2", compression='bz2', index=False)

In [19]:
# redefine what X and Y are - now with the zeros
X_train = flat_train
X_test = flat_test
y_train = X_train.pop(target_colname).values
y_test = X_test.pop(target_colname).values

This will take some time to finish with 6M rows.

In [20]:
automl_config = AutoMLConfig(task='forecasting',
                             debug_log='automl-grocery.log',
                             primary_metric='r2_score',
                             iterations=5,
                             X=X_train,   # Here the X_train and y_train have
                             y=y_train,   # explicit zeroes                             
                             n_cross_validations=2,
                             enable_ensembling=False,
                             path=project_folder,
                             verbosity=logging.INFO,    
                             **time_series_settings)

local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_eeca304c-d0cc-498d-a3cf-6907c5fc9b22
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating CV splits.


ERROR - Type: Client
Class: AutoMLPickleException
Message: Uploading X failed with Pickle error 
Traceback:
  File "E:\Anaconda3\envs\master_0509\lib\site-packages\azureml\automl\core\_vendor\automl\client\core\common\cache_store.py", line 382, in _upload
    self.pickler.dump(obj, path=path)
  File "E:\Anaconda3\envs\master_0509\lib\site-packages\azureml\automl\core\_vendor\automl\client\core\common\pickler.py", line 175, in dump
    raise AutoMLPickleException("Pickle error {}".format(e))



AutoMLPickleException: Pickle error 

Note we are not testing here because we pretend AutoML had a memory error. The data we are using in the tutorial may not actually be too large as we are only using items beginning with 'A'. But with the full dataset, the AutoML run in the above cell did fail.

# Technique 2: Splitting the datasets and composite models

## (NOT your usual train/test splitting)

In [ ]:
# Checkpoint: restarting the processing from saved files
# A good practice if you work with dataset sizes that crash python kernels
grain_colnames = ['Item', 'Site', 'Channel']
time_colname = 'SalesDate'
target_colname = 'Quantity'

import pandas as pd
flat_train = pd.read_csv("grocery_flat_trainvalid.csv.bz2", compression='bz2', parse_dates=[time_colname])
flat_test = pd.read_csv("grocery_flat_test.csv.bz2", compression='bz2', parse_dates=[time_colname])

We arrived at this point because AutoML had an out-of-memory event. 

If the data is too large, we need to split the series into multiple chunks to be handled separately. The data amounts in the individual chunks should ideally be close to equal.

It is best to split by the value of a specified column. It should be one of the grain columns - that way grains are guaranteed not to split across bins. 

'Item' seems like a good candidate column. How variable do the Item-based groups get in size?

In [ ]:
flat_train.groupby('Item').size()

We see the size is quite variable. Randomly assigning each item to a subset (chunk) would likely achieve approximately even chunks if the Items were about equal in data size, but we see the variation spans orders of magnitudes.

Therefore, after grouping by column value, then approximately solve the bin packing problem with maximum volume equal to the max number of rows we are willing to have for one model. This will guarantee near-optimal packing.  

In [ ]:
from tslib.ts_functions import split_into_chunks_by_size, split_into_chunks_by_groups
    
max_number_of_rows = 1 * 1000 * 1000;
train_frames, indices = split_into_chunks_by_size(flat_train, 'Item', max_number_of_rows)
# split test set using the same assignment of item to chunk (indices)
test_frames = split_into_chunks_by_groups(flat_test, 'Item', indices)

In [ ]:
# Checkpointing: pickle the split datasets
import pickle
pickle.dump( (train_frames, test_frames, indices), open('split_datasets.pkl', 'wb'))

In [ ]:
# and load them back after the lengthy procedure
import pickle
(train_frames, test_frames, indices) = pickle.load(open('split_datasets.pkl', 'rb'))

grain_colnames = ['Item', 'Site', 'Channel']
time_colname = 'SalesDate'
target_colname = 'Quantity'

Let's ensure we got decently sized chunks. Each chunk is a separate dataset that we will feed into AutoML.

In [ ]:
idx = 0
model_valid_for = indices[idx]
X_train = train_frames[idx].copy()
X_test = test_frames[idx].copy()

y_train = X_train.pop(target_colname).values
y_test = X_test.pop(target_colname).values

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

We see that the first chunk has the expected size for the train and test dataset. Let's test whether we messed up the datasets in the process of splitting. Running it through AutoML will run the data checks.

In [ ]:
automl_config = AutoMLConfig(task='forecasting',
                             debug_log='automl-grocery.log',
                             primary_metric='normalized_root_mean_squared_error',
                             iterations=5,
                             X=X_train,   # set the train to the NEW train set
                             y=y_train,   # which is now the first chunk only                      
                             n_cross_validations=2,
                             enable_ensembling=False,
                             path=project_folder,
                             verbosity=logging.INFO,    
                             **time_series_settings)

local_run = experiment.submit(automl_config, show_output=True)

In [ ]:
best_run, fitted_pipeline = local_run.get_output()
fitted_pipeline.steps
y_query = y_test.copy()
y_query.fill(np.nan)
y_fcst, X_trans = fitted_pipeline.forecast(X_test, y_query)   

In [ ]:
from tslib.ts_functions import align_outputs
df_all = align_outputs(y_fcst, X_trans, X_test, y_test)

That worked! Now that we were able to get a model, let's import the preferred and customary forecasting metrics and see how well we are doing.

In [ ]:
from tslib.ts_functions import MAPE, SMAPE, MAE

rmse = np.sqrt(mean_squared_error(df_all[target_column_name], df_all['predicted']))
print("[Test Data] \nRoot Mean squared error: %.2f" % rmse)
mae = mean_absolute_error(df_all[target_column_name], df_all['predicted'])
print('mean_absolute_error score: %.2f' % mae)
mape = MAPE(df_all[target_column_name], df_all['predicted'])
print('MAPE: %.2f' % mape)
smape = SMAPE(df_all[target_column_name], df_all['predicted'])
print('SMAPE: %.2f' % smape)
mae = MAE(df_all[target_column_name], df_all['predicted'])
print('MAE: %.2f' % mae)

The MAPE, and even SMAPE metrics can be deceiving when dealing with time series that are often close to zero. They divide by small values and are quite volatile. This makes them a poor fit for such problems. 

MAE is a better metric for these cases because it avoids division. But be careful because MAE results are unnormalized and comparing them only makes sense in reference to a fixed test set. MAE places weight on series whose absolute value is large. This is often correct in business. Mis-forecasting the Microsoft intern program travel expenses is a lot smaller deal than mis-forecasting the Azure datacenter operations cost. Series trained towards MAE should be of similar magnitude. 

It is often helpful to stratify the series by the `ceil(log(mean(series)))` and train separate models which allows for more homogeneous training set magnitude-wise and will lead to better performance on smaller-magnitude series.

But the best way to get a sense of how well a classifier is performing is to just plot the forecasts vs actuals. The mini-library has some plotting functions for you.

In [ ]:
# Let's do a time series plot for our difficult intermittent series
from tslib.ts_plot_utils import plot_forecast


#def plot_forecast(X_trainval, y_trainval,
#                  X_test, y_test, y_pred,
#                  target_column_name,
#                  time_column_name,
#                  actual_color='blue',
#                  pred_color='green',
#                  filter_dict = None)

filt = {'Item' : 'A0CA5B',
        'Site' : '1B3B',
        'Channel' : '811B' }

plot_forecast(X_train, y_train, X_test, y_test, y_fcst,
                  target_colname,
                  time_colname,
                  filter_dict = filt)

In [ ]:
# Let's do a scatter plot of actual vs predicted

%matplotlib notebook
test_pred = plt.scatter(df_all[target_column_name], df_all['predicted'], color='b')
test_actl = plt.scatter(df_all[target_column_name], df_all[target_column_name], color='g')
plt.legend((test_pred, test_actl), ('prediction', 'actual'), loc='upper left', fontsize=8)
plt.show()

### Putting the models back together

We split the data into small enough chunks that we can learn a model from each. But this is mildly inconvenient - we now have a collection of models and each of them can only process a subset of the data. We need to keep track of which models serves which `Item`s. To do this without writing awkward code on your side, we offer the general technique of *composite models*.

In the forecasting case, we usually have a natural splitting column in the grain. It would be extraordinary to have a single time series (no grain) big enough to require this treatment.

In the more general case of classification or regression, no good categorical splitting column may exist in the data. You can still split the data, and all the models you learn will be "universal" - capable of classifying any instance. You will lose some predictive power, because you learned from smaller datasets. But you can get it back by running the example through `predict()` of *all* the models and ensembling the predictions (just take a simple majority vote or average).


In [ ]:
class CompositeModel:    
    
    def __init__(self, split_column, target_column):
        self._split_column = split_column
        self._target_column = target_column
        self._item_run_map = dict()
        self._item_model_map = dict()
        self._model_impls = dict()
        self._indices = dict()
    

    # todo: redo this so that this class splits its own training data
    # on the splitcolumn, ideally in sequence, rather than materializing
    # the whole dataset again in memory as a sequence of chunks. 
    
    def fit(self, train_frames, indices):    
        
        self._indices = indices
            
        # TODO: this is hard-wired and overwrites outer scope        
        time_series_settings = {
            'time_column_name': time_colname,
            'grain_column_names': grain_colnames,
            'drop_column_names': [],
            'max_horizon': 31
        }
        
        for idx, Xy in enumerate(train_frames):
            
            if len(Xy) == 0:
                print('Warning: found a zero-length frame at index ' + str(idx))
                continue
            
            X_train = Xy.copy()            
            y_train = X_train.pop(self._target_column).values                        
          
            automl_config = AutoMLConfig(task='forecasting',
                                debug_log='automl-grocery.log',
                                primary_metric='normalized_root_mean_squared_error',
                                iterations=5,
                                X=X_train,
                                y=y_train,                             
                                n_cross_validations=3,
                                enable_ensembling=False,
                                path=project_folder,
                                verbosity=logging.INFO,    
                                **time_series_settings)
        
            # get the model and metadata
            local_run = experiment.submit(automl_config, show_output=True) # Parent run 
            best_run, fitted_pipeline = local_run.get_output()             # Favorite child
            model_id = best_run.id
            print('Learned model ' + str(model_id))  # this is not working - needs a different ID
        
            # record the model for item
            self._model_impls[model_id] = fitted_pipeline
            for idx, item in enumerate(self._indices[idx]):
                self._item_model_map[item] = model_id
                self._item_run_map[item] = best_run.id
                
                
    def forecast(self, X_test, y_test):
        
        # split X and y together by splitcolumn
        X_copy = X_test.copy()
        X_copy['__automl_target_column'] = y_test
        chunks = split_into_chunks_by_groups(X_copy, self._split_column, self._indices)
        
        ys = []
        X_transes = []
        for chunk in chunks:
            
            # skip potentially empty splits
            if len(chunk) == 0:
                continue
            
            # Look up the right model. It should be the same model 
            # for the whole chunk by construction
            item = chunk.loc[chunk.index[0], self._split_column]
            modelid = self._item_model_map[item]
            print('Using model ' + str(modelid))
            model = self._model_impls[modelid]
            
            paranoid = True
            if paranoid:
                for item2 in pd.unique(chunk[self._split_column]):
                    assert(
                           (item2 in self._item_model_map.keys()) and 
                           (self._item_model_map[item2] == modelid),
                           'Item ' + str(item2) + ' is not mapped to the same model ' + str(modelid) + ' as ' + str(item)
                    )
                           
            y_chunk = chunk.pop('__automl_target_column').values
            y_pred, X_trans = model.forecast(chunk, y_chunk)
            ys.append(y_chunk)
            X_transes.append(X_trans)
            
        return np.concatenate(ys), pd.concat(X_transes)                    

In [ ]:
cm = CompositeModel(split_column = 'Item', target_column = 'Quantity')
cm.fit(train_frames, indices)

In [ ]:
# test the forecasts on one test dataframe
X_test0 = test_frames[0].copy()
y_test0 = X_test0.pop(target_colname).values
y_test0.fill(np.nan)
r0 = cm.forecast(X_test0, y_test0)

In [ ]:
# Create the full test set, actuals, and query

# OK to copy the smaller test frames
ys = [ X[target_colname].values for X in test_frames ]
Xs = [ X.copy().drop(columns=[target_colname]) for X in test_frames]
X_test = pd.concat(Xs)
y_test = np.concatenate(ys)
y_query = y_test.copy()
y_query.fill(np.nan)

In [ ]:
# test on all chunks
y_fcst, X_trans = cm.forecast(X_test,y_test)
df_all = align_outputs(y_fcst, X_trans, X_test, y_test)
df_all.head()

In [ ]:
rmse = np.sqrt(mean_squared_error(df_all[target_column_name], df_all['predicted']))
print("[Test Data] \nRoot Mean squared error: %.2f" % rmse)
mae = mean_absolute_error(df_all[target_column_name], df_all['predicted'])
print('mean_absolute_error score: %.2f' % mae)
mape = MAPE(df_all[target_column_name], df_all['predicted'])
print('MAPE: %.2f' % mape)
smape = SMAPE(df_all[target_column_name], df_all['predicted'])
print('SMAPE: %.2f' % smape)

# Technique 3: Target transforms

This dataset is extremely challenging due to the abundance of zero values. Also, the forecasts sometimes venture below zero, which will not make sense to business customers. In those cases, it often helps to transform the target value with something like a log-transform.

In [ ]:
import math

def log_transform(nparr):
    return nparr.apply(lambda x: math.log(x+1))
    

def log_inverse_transform(nparr):
    """
    With zero truncation so not a perfect inverse
    """
    return nparr.apply(lambda y: max(0, math.exp(y) - 1))
    
# For example, 
log_inverse_transform(log_transform(y_train[0:5]))

In [ ]:
idx = 0
model_valid_for = indices[idx]
X_train = train_frames[idx].copy()
X_test = test_frames[idx].copy()

# NOTE log transform
y_train = log_transform(X_train.pop(target_colname).values)
y_test = X_test.pop(target_colname).values

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
automl_config = AutoMLConfig(task='forecasting',
                             debug_log='automl-grocery.log',
                             primary_metric='normalized_root_mean_squared_error',
                             iterations=5,
                             X=X_train,
                             y=y_train,                             
                             n_cross_validations=2,
                             enable_ensembling=False,
                             path=project_folder,
                             verbosity=logging.INFO,    
                             **time_series_settings)

local_run = experiment.submit(automl_config, show_output=True)

In [ ]:
best_run, fitted_pipeline = local_run.get_output()
fitted_pipeline.steps
y_query = y_test.copy()
y_query.fill(np.nan)
y_fcst, X_trans = fitted_pipeline.forecast(X_test, y_query)   

In [ ]:
filt = {'Item' : 'A0CA5B',
        'Site' : '1B3B',
        'Channel' : '811B' }

# TESTME: do we need to plot from the aligned data frame?
plot_forecast(X_train, y_train, X_test, y_test, 
                  inverse_log_transform(y_fcst),       # NOTE inverse transform
                  target_colname,
                  time_colname,
                  filter_dict = filt)

# Caveat 4: A note on pandas indexes

One problem that often occurs with dataframes arising from filtering is that *level names* are retained in the index when the values are *removed*. This can create empty groups when grouping by index. 

In [ ]:
# This problem is pandas version dependent (TODO: which versions?)
print(pd.__version__)

ex_df = pd.DataFrame({'Item' : ['A', 'A', 'B', 'B', 'C'], 
                       'Day' : ['Mon', 'Tue', 'Mon', 'Tue', 'Mon'],
                       'Sales' : [1, 2, 3, 4, 5]})
ex_df.set_index(['Item'], inplace=True)
ex_df = ex_df[ex_df['Day'] == 'Tue']  # filter down to Tue, leaving out 'C'
# ex_df.index.get_level(level='Item')
ex_df.groupby(level='Item').sum()

# TODO: make this problem actually show up with the wrong pandas version.


In [ ]:
# the easiest fix is to rebuild the dataframe from csv
ex_df.reset_index().to_csv('temp_ex_df.csv.bz2', index=False)
ex_df = pd.read_csv('temp_ex_df.csv.bz2')

# or use StringIO if you having multiple data copies in memory is acceptable

This causes hard-to-understand errors, so let's make sure it didn't happen to us, checking ALL the data chunks.

In [ ]:
# check for empty levels

def empty_grains(df, grain_colnames):
    """"
    returns the list of empty grains
    """
    empties = df.groupby(grain_colnames).filter(lambda g: len(g) == 0)
    return empties

frames_with_empties = [empty_grains(df, grain_colnames).shape[0] > 0 for df in train_frames]

In [ ]:
any(frames_with_empties)

Note: This is *different* from checking whether some of the training data frames are actually empty. That would look like:

In [ ]:
# check for empty dataframes in train_frames
[idx for idx, df in enumerate(train_frames) if len(df)==0]

# Conclusions: What is missing here?

This exercise does not address a few things. But you are now equipped to build them on your own.
* Deployment of composite models
 * The deployment procedure will need to register all the models in the dictionary.
 * The indices dictionary will need to be deployed to the docker image in pickled form.
 * The template script `init` function will load the indices into memory.
 * The template script `run` function will need lazy loading of the models from the workspace.
* Those orphaned short series.
 * We left a number of series behind when we decided they were "too short for ML". That is quite some hubris;
   chances are the business owners do need those forecasts. 
  * We should therefore create another wraparound model which will provide forecasts for them. 
    High accuracies cannot be expected from such data, and a simple model will do. 
    In the pattern of the composite model, we could keep a dictionary of retained parameters
    of say, an exponential smoothing model, for each grain in the dataset.
* Info schedule issues
 * TODO: write the function